# Cosine Name Matching using Bigrams
<b> Emilio Ramos Monzalvo - 09/01/2021 </b>

## Config

In [1]:
%load_ext autotime

In [2]:
import pathlib
import string
import sys
from tqdm import tqdm

import numpy as np
import pandas as pd
import random

import re
from sklearn.feature_extraction.text import TfidfVectorizer

from scipy.sparse import csr_matrix
import sparse_dot_topn.sparse_dot_topn as ct  # cosine matching

from fuzzywuzzy import fuzz
import fuzzywuzzy.process as fuzz_process

### Global Variables

In [3]:
NGRAMS = 2

## Read in Data

In [4]:
df = pd.read_csv(
            'Data/prob_race_given_surname_2010.csv',
            index_col=False,
            na_values=[''],
            keep_default_na=False,
        )[['name']]
print('Number of Rows: ', len(df))
df.head()

Number of Rows:  162254


,name
0,AAB
1,AABERG
2,AABY
3,AADLAND
4,AAFEDT


## Pre-Process

### Normalize Names

In [5]:
def preprocess_names(names: pd.Series) -> pd.Series:
        """Take names and run a normalization routine"""
        
        # Make a transalation table of unwanted characers
        unwanted_characters = (
            string.digits +
            string.punctuation +
            string.whitespace
        )
        
        # Remove unwanted characters efficiently
        translation_table =  str.maketrans('', '', unwanted_characters)
        
        # Run our string operations (remember NAN is a valid name)
        output = (
            names.fillna('')
                 .astype(str)
                 .str.translate(translation_table)
                 .str.upper()
                 .str.replace(r'\s?J\.*?R\.*\s*?$', '', regex=True)
                 .str.replace(r'\s?S\.*?R\.*\s*?$', '', regex=True)
                 .str.replace(r'\s?III\s*?$',      '', regex=True)
                 .str.replace(r'\s?IV\s*?$',       '', regex=True)
        )
        output.name = 'name'
        
        return output

df.name = preprocess_names(df.name)

### Remove Duplicates

In [6]:
df = df.drop_duplicates(subset=['name']).reset_index(drop=True)
print('Unique Names: ', len(df))

Unique Names:  162250


### Transform Name to Title

In [7]:
df['name_title'] = df.name.str.title()

### Split into Train and Test

In [8]:
df['train_or_test'] = np.random.choice(a=['test', 'train'], size=len(df), p=[.2, .8], replace=True)

df_train = df.loc[df['train_or_test']=='train'].reset_index(drop=True)
df_test = df.loc[df['train_or_test']=='test'].reset_index(drop=True)

df['train_or_test'].value_counts(normalize=True)

train    0.801818
test     0.198182
Name: train_or_test, dtype: float64

### TFIDF Vectorization

In [9]:
# Vectorize using TfidVectorizer and Ngrams
vect = TfidfVectorizer(analyzer='char', max_df=0.3, min_df=3, ngram_range=(NGRAMS, NGRAMS), lowercase=False)
tf_idf_matrix_train = vect.fit_transform(df.loc[df['train_or_test']=='train'].name_title)
tf_idf_matrix_test = vect.transform(df.loc[df['train_or_test']=='test'].name_title)
vocab = vect.vocabulary_
len(vocab)

958

## Find Similarities

### Cosine Similarity

In [10]:
def cosine_similarity_top(A: csr_matrix, B: csr_matrix, topn: int = 10, lower_bound: int=0) -> csr_matrix:
    # force A and B as a CSR matrix.
    # If they have already been CSR, there is no overhead
    A = A.tocsr()
    B = B.tocsr()
    M, _ = A.shape
    _, N = B.shape
 
    idx_dtype = np.int32
 
    nnz_max = M*topn
 
    indptr = np.zeros(M+1, dtype=idx_dtype)
    indices = np.zeros(nnz_max, dtype=idx_dtype)
    data = np.zeros(nnz_max, dtype=A.dtype)
    
    ct.sparse_dot_topn(
            M, N, np.asarray(A.indptr, dtype=idx_dtype),
            np.asarray(A.indices, dtype=idx_dtype),
            A.data,
            np.asarray(B.indptr, dtype=idx_dtype),
            np.asarray(B.indices, dtype=idx_dtype),
            B.data,
            topn,
            lower_bound,
            indptr, indices, data)
    
    return csr_matrix((data,indices,indptr),shape=(M,N))

In [44]:
#  Run the optimized cosine similarity function. 
#  Only stores the top 10 most similar items with a similarity above 0.8
matches = cosine_similarity_top(A=tf_idf_matrix_train, 
                                B=tf_idf_matrix_test.transpose(), 
                                topn=50, lower_bound=0)
matches.size

6502023

### Get Best Matches

In [64]:
def get_matches_df(sparse_matrix: csr_matrix, name_vector_a: pd.Series, name_vector_b: pd.Series , topn: int=-1) -> pd.DataFrame:
    """Unpack Sparse Matrix Matches into a Data Frame"""
    
    non_zeros = sparse_matrix.nonzero()
    
    sparserows = non_zeros[0]
    sparsecols = non_zeros[1]
    
    if topn > 0:
        nr_matches = topn
    else:
        nr_matches = sparsecols.size
    
    left_side = np.empty([nr_matches], dtype=object)
    right_side = np.empty([nr_matches], dtype=object)
    similairity = np.zeros(nr_matches)

    for index in tqdm(range(0, nr_matches), desc="Computing Similarities..."):
        left_side[index] = name_vector_a[sparserows[index]]
        right_side[index] = name_vector_b[sparsecols[index]]
        similairity[index] = np.floor(sparse_matrix.data[index]*100)
    
    return pd.DataFrame({'name': right_side,
                          'match': left_side,
                           'match_score': similairity})

In [78]:
# store the  matches into new dataframe called matched_df and printing 10 samples
cosine_match = get_matches_df(sparse_matrix=matches,
                            name_vector_a=df_train.name, 
                            name_vector_b=df_test.name,
                            topn=-1)

cosine_match = cosine_match.loc[((cosine_match['match_score'] < 99) & 
                                 (cosine_match['name'].isin(df_test.name)))]\
                            .sort_values(['name', 'match_score'], ascending=[True, False])\
                            .reset_index(drop=True) # For removing all exact matches

cosine_match.groupby(['name']).head(1).head(20)

Computing Similarities...: 100%|██████████████████████████████████████████| 6502023/6502023 [01:15<00:00, 85619.21it/s]


,name,match,match_score
0,AAB,AABY,83.0
35,AADLAND,AALAND,77.0
150,AAKRE,AAKER,69.0
252,AALGAARD,AAGAARD,80.0
363,AAMIR,AAS,65.0
436,AAMOT,AAMOLD,78.0
508,AARDEMA,AAGARD,68.0
558,AARESTAD,AANESTAD,86.0
658,AARONS,AARON,92.0
698,AASLAND,AALAND,78.0


## Fuzzy Distance

In [79]:
def get_fuzzy_matches(old_names: pd.Series, new_names: pd.Series, cosine_match: pd.DataFrame, topn: int = 1, min_score:int = 0) -> pd.DataFrame:
    """Returns the topn best matches for each of the names in vector b
        
        old_names: Existing Names
        new_names: New Names that need Matching
        topn: number of results for each new name
    
    """
    
    fuzzy_matches = pd.DataFrame(columns=['name', 'match', 'match_score'])
    
    res_idx = 0
    for new_name in tqdm(new_names.values, desc="Computing Similarities..."):
        
        # Use subset of cosine names
        most_like_names = cosine_match.loc[cosine_match['name'] == new_name].match.values
        
        # Check if a there is a subset of best names; otherwise, compare against all old names.
        if len(most_like_names) == 0:
            fuzz_matches = fuzz_process.extract(new_name, old_names.values, scorer=fuzz.token_sort_ratio)
        else:
            fuzz_matches = fuzz_process.extract(new_name, most_like_names, scorer=fuzz.token_sort_ratio)
            
        if len(fuzz_matches) > 0:
            for match in fuzz_matches:
                fuzzy_matches.loc[res_idx] = {'name':new_name, 'match':match[0], 'match_score':match[1]}
                res_idx += 1
        else:
            fuzzy_matches.loc[res_idx] = {'name':new_name, 'match':'', 'match_score':0}
            
    return fuzzy_matches.loc[fuzzy_matches['match_score'] > min_score].sort_values(['name', 'match_score'], ascending=[True, False]).groupby(['name']).head(topn)

In [80]:
fuzzy_match = get_fuzzy_matches(old_names=df_train.name, new_names=df_test.head(2000).name, cosine_match=cosine_match, topn=1, min_score=0)
fuzzy_match.head(20)

Computing Similarities...: 100%|███████████████████████████████████████████████████| 2000/2000 [13:35<00:00,  2.45it/s]


,name,match,match_score
0,AAB,AABY,86
5,AADLAND,AALAND,92
10,AAKRE,AKRE,89
15,AALGAARD,AAGAARD,93
20,AAMIR,ALAMIRI,83
25,AAMOT,AAMODT,91
30,AARDEMA,AARDSMA,86
35,AARESTAD,AARSTAD,93
40,AARONS,AARON,91
45,AASLAND,AALAND,92


## Compare Best Matches

In [81]:
matches_df = cosine_match.groupby(['name']).head(1).merge(fuzzy_match.groupby(['name']).head(1), on=['name'], suffixes=('_cosine', '_fuzzy'), how='inner')

In [82]:
matches_df.loc[matches_df['match_cosine'] != matches_df['match_fuzzy']]

,name,match_cosine,match_score_cosine,match_fuzzy,match_score_fuzzy
2,AAKRE,AAKER,69.0,AKRE,89
4,AAMIR,AAS,65.0,ALAMIRI,83
5,AAMOT,AAMOLD,78.0,AAMODT,91
6,AARDEMA,AAGARD,68.0,AARDSMA,86
7,AARESTAD,AANESTAD,86.0,AARSTAD,93
...,...,...,...,...,...
1992,BAYLY,BAYLE,80.0,BALY,89
1993,BAYNARD,MAYNARD,85.0,BAYARD,92
1994,BAYNHAM,BAYNE,68.0,BAYHAM,92
1996,BAYOH,BAYOT,77.0,BAYOUTH,83


## Backup

### Get Word List and Frequency

In [17]:
# # sort n-gram by freq (highest -> lowest)
# words = []
# for b in vocab:
#     c = vocab[b]
#     words.append((tf_idf_matrix[:, c].sum(), b))
# words = sorted(words, reverse=True)
# words_list = [w[1] for w in words]
# num_words = len(words_list)
# print("num_words = %d" % num_words)

### Convert Data to Ngrams for Modeling

In [18]:
# from tensorflow.keras.preprocessing.sequence import pad_sequences

In [19]:
# def find_ngrams(text, n):
#     a = zip(*[text[i:] for i in range(n)])
#     wi = []
#     for i in a:
#         w = ''.join(i)
#         try:
#             idx = words_list.index(w)
#         except:
#             idx = 0
#         wi.append(idx)
#     return wi

In [20]:
# # build X from index of n-gram sequence
# X = np.array(df.name_title.apply(lambda c: find_ngrams(c, NGRAMS)))

# # check max/avg feature
# X_len = []
# for x in X:
#     X_len.append(len(x))

# max_feature_len = max(X_len)
# avg_feature_len = int(np.mean(X_len))
# del(X_len)

# print("Max feature len = %d, Avg. feature len = %d" % (max_feature_len, avg_feature_len))

### Add Padding

In [21]:
# max_features = num_words # 20000
# if max_feature_len < 20:
#     feature_len = 20 # avg_feature_len # cut texts after this number of words (among top max_features most common words)
# else:
#     feature_len = max_feature_len + 5

# print(len(X), 'name sequences')

# print('Pad sequences (samples x time)')
# X_pad = pad_sequences(X, maxlen=feature_len)
# print('X shape:', X_pad.shape)